<a href="https://colab.research.google.com/github/bae3559/2021summerintern/blob/main/ResNet_withTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ResNet 

```
torchvision.models.resnet
```

- resnet(18,34,50,101,152)를 다 만들 수 있도록 되어있음.
- 3x224x224 입력을 기준으로 만들도록 되어있음.
- input size가 다른 경우 ResNet를 적용하려면 어떻게 할 수 있을지 생각해보기. 


In [7]:
## ResNet  


import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [9]:
class BasicBlock(nn.Module):
    def __init__(self,inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()

        self.conv1 = conv3x3(inplanes, planes, stride)  # stride 값을 받은 걸 씀 
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes,planes) # stride 무조건 1
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):

      # if x.shape = 3x64x64
      identity = x   #identity.shape = 3x64x64
      # if stride=2  
      out = self.conv1(x) # padding :1 , out.shape = 3x32x32
      out = self.bn1(out)  # out.shape = 3x32x32
      out = self.relu(out)  # out.shape = 3x32x32

      out = self.conv2(out) # out.shape = 3x32x32
      out = self.bn2(out)  # out.shape = 3x32x32

      # 이렇게 되면 지금 identity 랑 out의 shape이 달라서 더 하는게 불가능해진다. 
      # 그럴 때, downsample을 해주면 됨! 

      if downsample != None:
        identity = self.downsample(x)

      out += identity 
      out = self.relu(out)

      return out

In [26]:
class Bottleneck(nn.Module):

    expansion = 4
    def __init__(self, inplanes, planes, stride=1, downsample=None):
      super(Bottleneck, self).__init__()
      
      self.conv1 = conv1x1(inplanes, planes)
      self.bn1 = nn.BatchNorm2d(planes)
      self.conv2 = conv3x3(planes, planes,stride)
      self.bn2 = nn.BatchNorm2d(planes)
      self.conv3 = conv1x1(planes, planes * self.expansion)
      self.bn3 = nn.BatchNorm2d(planes * self.expansion)

      self.downsample = downsample
      self.relu = nn.ReLU(inplace=True)
      self.stride = stride

    def forward(self, x):

      identity = x

      out = self.conv1(x)
      out = self.bn1(out)
      out = self.relu(out)

      out = self.conv2(out)
      out = self.bn2(out)
      out = self.relu(out)

      out = self.conv3(out)
      out = self.bn3(out)
      
      if downsample != None:
        identity = self.downsample(x)

      out += identity 
      out = self.relu(out)

      return out

In [15]:
def resenet18(pretrained=False, **kwargs):

  """
    Args:
        pretrained(bool): If True, returns a model pre-trained on ImageNet
  """
  model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs) # resnet18의 경우 basic block 하나에 conv가 2개
  # 계산 해보면 2*(2+2+2+2) = 16  맨앞에 conv1, 맨 뒤에 fc  그래서 총 18개
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
  return model

def resnet152(pretrained=False, **kwargs):
  model = ResNet(Bottleneck,[3, 8, 36, 3], **kwargs) 
  # Bottleneck의 경우 conv가 총 3개, 1(ocnv1)+3*(3+8+36+3)+1(fc) = 152
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
  return model

In [28]:
class ResNet(nn.Module):
  #model = ResNet(Bottleneck,[3, 4, 6, 3], **kwargs) resnet50
  def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
    super(ResNet, self).__init__()
    self.inplanes = 64
    
    #inputs = 3x224x224
    self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False) 
    # outputs = self.conv1(inputs)라면, outputs.shape = [64x112x112]
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    #  64x56x56 
    # layers = [3,4,6,3] 
    self.layer1 = self._make_layer(block, 64, layers[0])
    self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
    self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
    self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
    
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    # weight 초기화 
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight,1)
        nn.init.constant_(m.bias,1)
    # Zero-Initialize the last BN in eash residual branch, 
    # So that the residual branch starts with zeros, and each residual block behave like an identity.
    # This improves the model by 0.2~0.3% encoding to https://arxiv.org/abs/1706.02677 (수행의 근거가 적혀있는 논문)
    if zero_init_residual:
      for m in self.modules():  
        if isinstance(m, Bottleneck):
          nn.init.constant_(m.bn2.weight,0)
        elif isinstance(m, BasicBlock):
          nn.init.constant_(m.bn2.weight,0)


  #self.layer1 = self._make_layer(block=Bottleneck, 64, 3) 
  #self.layer2 = self._make_layer(Bottleneck, planes=128, blocks=4, stride=2)
  def _make_layer(self, block, planes, blocks, stride=1):
    downsample = None
    # downsample의 경우 stride가 2일 때, feature size가 줄어드니까, identity꺼도 낮춰주려고 사용하는 건데, 여기서는 더불어서 channel size 맞춰주는 용도로도 쓴다! 
    # 뭐 결국 크기가 안 맞으면 크기가 맞게 바꿔 주는 거 
    if stride != 1 or self.inplanes != planes * block.expansion: # self.inplanes = 64 != 64*4 = 256 
      downsample = nn.Sequential( 
          conv1x1(self.inplanes, planes*block.expansion, stride),  # conv1x1(64, 256, stride=1) 
          # conv1x1(256,512,2) 
          nn.BatchNorm2d(planes*block.expansion) ) # batchnorm2d (256)
          # batchnorm2d(512)
      layers=[]
      layers.append(block(self.inplanes, planes, stride, downsample)) 
      # Bottleneck(64, 64, 1, downsample)
      # Bottleneck(256,128,2, downsample(conv1x1(256,512,2)))
      self.inplanes = planes*block.expansion # 위에서 이제 크기를 맞춰 줬으니 inplanes도 256으로 업데이트 ! 
      # 이제 self.inplanes = 512
      for _ in range(1, blocks):
      # for _ in range(1, 3) sequential이 2개 더 append 되는거! 그래서 총 3개!
      # for _ in range(1, 4) sequential이 3개 더 append 
        layers.append(block(self.inplanes, planes))

      # self.layer1 = [ Bottleneck(64, 64, 1, downsample)
      #                 Bottleneck(256,64)
      #                 Bottleneck(256,64)]
      # self.layer2 = [Bottleneck(256,128,2, downsample)
      #                 Bottleneck(512,128)
      #                 Bottleneck(512,128)
      #                 Bottleneck(512,128)]
      return nn.Sequential(*layers)


  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x) #global average pooling 
    x = x.view(x.size(0), -1) 
    x = self.fc(x)

    return x


In [29]:
res = resnet50()
res

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): Batc

0
